In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import expipe
import pathlib
import numpy as np
import spatial_maps.stats as stats
import septum_mec
import septum_mec.analysis.data_processing as dp
import septum_mec.analysis.registration
import head_direction.head as head
import spatial_maps as sp
import speed_cells.speed as spd
import re
import joblib
import multiprocessing
import shutil
import psutil
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
# import seaborn as sns
from distutils.dir_util import copy_tree
from neo import SpikeTrain
import scipy

from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()

from septum_mec.analysis.statistics import load_data_frames, make_paired_tables, make_statistics_table

from spikewaveform.core import calculate_waveform_features_from_template, cluster_waveform_features

from septum_mec.analysis.plotting import violinplot, despine

In [3]:
%matplotlib inline
plt.rc('axes', titlesize=12)
plt.rcParams.update({
    'font.size': 12, 
    'figure.figsize': (6, 4), 
    'figure.dpi': 150
})

output_path = pathlib.Path("output") / "cell-count"
(output_path / "statistics").mkdir(exist_ok=True, parents=True)
(output_path / "figures").mkdir(exist_ok=True, parents=True)
output_path.mkdir(exist_ok=True)

In [4]:
data, labels, colors, queries = load_data_frames()

Number of sessions above threshold 194
Number of animals 4
Number of individual gridcells 139
Number of gridcell recordings 230


In [7]:
cell_types = [
    'gridcell',
    'ns_inhibited', 
    'ns_not_inhibited',
    'bs',
    'bs_not_gridcell'
]

In [9]:
data['cell_type'] = np.nan
for cell_type in cell_types:
    data.loc[data.eval(cell_type), 'cell_type'] = cell_type

In [15]:
data.drop_duplicates('unit_id').groupby(['entity', 'cell_type']).count()

action  baseline  frequency    i   ii  session  \
entity cell_type                                                          
1833   bs                    89        89         34   89   89       89   
       bs_not_gridcell      166       166         89  166  166      166   
       ns_inhibited          12        12         12   12   12       12   
       ns_not_inhibited      56        56         24   56   56       56   
1834   bs                    14        14          7   14   14       14   
       bs_not_gridcell      216       216        111  216  216      216   
       ns_inhibited           4         4          4    4    4        4   
       ns_not_inhibited       7         7          3    7    7        7   
1839   bs                    18        18          8   18   18       18   
       bs_not_gridcell       70        70         47   70   70       70   
       ns_inhibited          10        10         10   10   10       10   
       ns_not_inhibited       6         6          3    6    6        6   
1849   bs                     8         8          5    8    8        8   
       bs_not_gridcell      229       229        105  229  229      229   
       ns_inhibited           4         4          4    4    4        4   
       ns_not_inhibited      27        27         10   27   27       27   

                         stim_location  stimulated  tag  date  ...  bs_stim  \
entity cell_type                                               ...            
1833   bs                           34          89   89    89  ...       34   
       bs_not_gridcell              89         166  166   166  ...       89   
       ns_inhibited                 12          12   12    12  ...       12   
       ns_not_inhibited             24          56   56    56  ...       24   
1834   bs                            7          14   14    14  ...        7   
       bs_not_gridcell             111         216  216   216  ...      111   
       ns_inhibited                  4           4    4     4  ...        4   
       ns_not_inhibited              3           7    7     7  ...        3   
1839   bs                            8          18   18    18  ...        8   
       bs_not_gridcell              47          70   70    70  ...       46   
       ns_inhibited                 10          10   10    10  ...       10   
       ns_not_inhibited              3           6    6     6  ...        3   
1849   bs                            5           8    8     8  ...        5   
       bs_not_gridcell             105         229  229   229  ...      104   
       ns_inhibited                  4           4    4     4  ...        4   
       ns_not_inhibited             10          27   27    27  ...       10   

                         bs_ctrl  ns_inhibited  ns_not_inhibited  gridcell  \
entity cell_type                                                             
1833   bs                     55            89                89        89   
       bs_not_gridcell        77           166               166       166   
       ns_inhibited            0            12                12        12   
       ns_not_inhibited       32            56                56        56   
1834   bs                      7            14                14        14   
       bs_not_gridcell       105           216               216       216   
       ns_inhibited            0             4                 4         4   
       ns_not_inhibited        4             7                 7         7   
1839   bs                     10            18                18        18   
       bs_not_gridcell        23            70                70        70   
       ns_inhibited            0            10                10        10   
       ns_not_inhibited        3             6                 6         6   
1849   bs                      3             8                 8         8   
       bs_not_gridcell       123           229               229       229   
   

In [28]:
table = pd.DataFrame()
table['Gridcell'] = data.drop_duplicates('unit_idnum').query('gridcell').groupby('entity')['action'].count()
table['BS not gridcell'] = data.drop_duplicates('unit_idnum').query('bs_not_gridcell').groupby('entity')['action'].count()
table['NSi'] = data.drop_duplicates('unit_idnum').query('ns_inhibited').groupby('entity')['action'].count()
table['NSni'] = data.drop_duplicates('unit_idnum').query('ns_not_inhibited').groupby('entity')['action'].count()
table

,Gridcell,BS not gridcell,NSi,NSni
entity,,,,
1833,94,165,16,52
1834,14,216,4,7
1839,19,70,11,5
1849,8,229,8,23


In [30]:
table.to_latex(output_path / "statistics" / "counts.tex")
table.to_csv(output_path / "statistics" / "counts.csv")

# Store results in Expipe action

In [31]:
action = project.require_action("cell-count")

In [32]:
copy_tree(output_path, str(action.data_path()))

['/media/storage/expipe/septum-mec/actions/cell-count/data/statistics/counts.tex',
 '/media/storage/expipe/septum-mec/actions/cell-count/data/statistics/counts.csv']

In [33]:
septum_mec.analysis.registration.store_notebook(action, "20_cell_count.ipynb")